In [2]:
import time
import os
import sys
sys.path.append("../dataset")
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss
import numpy as np
#from ...dataset.multi_label_dataset import MultiLabelDataset

from multi_label_dataset import MultiLabelDataset

In [3]:

seeg_file = '../../data/new_downsampled_seeg.npy'
label_folder = '../../data/presence_of_faces/'
train_ratio = 0.7
test_ratio = 0.15
train_dataset =MultiLabelDataset(seeg_file=seeg_file, label_folder=label_folder, split='train', train_ratio=train_ratio,
                                    test_ratio=test_ratio)
val_dataset = MultiLabelDataset(seeg_file=seeg_file, label_folder=label_folder, split='val', train_ratio=train_ratio,
                                    test_ratio=test_ratio)
test_dataset = MultiLabelDataset(seeg_file=seeg_file, label_folder=label_folder, split='test', train_ratio=train_ratio,
                                    test_ratio=test_ratio)



Initialized train dataset with 5144 samples
Initialized val dataset with 1103 samples
Initialized test dataset with 1103 samples


In [4]:
seeg_train, label_train = train_dataset.seeg_data, train_dataset.label_data
seeg_test, label_test = test_dataset.seeg_data, test_dataset.label_data


### Load the sEEG data, its shape is (number_of_electrode, samples_recorded), (84, 7526347)

### A single second of SEEG data contains 1024 samples. We will segment the SEEG data into one-second intervals. Any portion of the SEEG data that does not comprise a complete one-second segment will be excluded from analysis.

### Truncate the face labels to match the duration of sEEG data

### Perform SVM  to do the classfication between characters 
In order to compare the accuracy with DNN, we also split the data into 70% traning, 15% testing and 15% validation, although 15% validation is not used.

In [19]:
# First split: 70% training, 30% temp (which will be split into test and validation)
X_train, X_temp, y_train, y_temp = train_test_split(flattened_data, truncated_face_labels.T, test_size=0.3, random_state=42)

# Second split: Split the temp data into test and validation (50% each of the temp data)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Now, X_train, y_train are our training sets (70% of the data)
# X_test, y_test are our test sets (15% of the data)
# X_val, y_val are our validation sets (15% of the data)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [5]:
X_train = seeg_train.reshape(seeg_train.shape[0],-1)
y_train = label_train
X_test = seeg_test.reshape(seeg_test.shape[0],-1)
y_test = label_test

In [6]:
X_test.shape

(1103, 86016)

In [7]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:

# Choose a base estimator
base_estimator = SVC(probability=True,class_weight ="balanced")

# OneVsRestClassifier
ovr_classifier = OneVsRestClassifier(base_estimator)

# Train the model
ovr_classifier.fit(X_train, y_train)

# Predictions
y_pred = ovr_classifier.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
hamming = hamming_loss(y_test, y_pred)

print("Accuracy:", accuracy)
print("Hamming Loss:", hamming)
